<a href="https://colab.research.google.com/github/ckz2011/MeetingNotes/blob/main/meeting_transcript_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: Meeting Transcript Summarizer & Action-Item Extractor

This notebook implements a meeting transcript summarizer and action-item extractor using GPT-4o or GPT-4.1-mini.


## Step 1: Install Required Libraries
Make sure you have `openai` and `pydantic` installed. Uncomment and run the cell below if needed.

In [ ]:
 !pip install openai pydantic

In [ ]:
!pip install python-dotenv

## Step 2: Define the Prompt Template and Extraction Function

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List


env_variables = """
AZURE_OPENAI_ENDPOINT=https://eastus2.api.cognitive.microsoft.com/
AZURE_OPENAI_API_KEY=6367eda5b6ea49b7a64f078f38b392b1
OPENAI_API_VERSION=2024-12-01-preview
LANGFUSE_SECRET_KEY=sk-lf-4cb9921-317d-45db-b918-4d033774bfd3
LANGFUSE_PUBLIC_KEY=pk-lf-a8a99717-832e-46fe-8b20-651d7fb37889
OPENWEATHERMAP_API_KEY=6a8b0ac166a37e2b7a38e64416b3c3fe
"""

# Create the .env file in the current directory
with open('.env', 'w') as f:
    f.write(env_variables)



from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the environment variables
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_version = os.getenv('OPENAI_API_VERSION')


# Print to verify
print(f"Azure OpenAI Endpoint: {azure_openai_endpoint}")
print(f"OpenWeatherMap API Key: {openweathermap_api_key}")


from openai import AzureOpenAI
client = AzureOpenAI()


class MeetingNotes(BaseModel):
    summary: str
    action_items: List[str]

def extract_meeting_notes(transcript: str) -> dict:
    prompt = (
        "Given the following meeting transcript, extract:\n"
        "1. A two-sentence summary of the meeting.\n"
        "2. A list of all action items mentioned, each prefixed with a dash (-).\n"
        "Return the result as a JSON object with keys 'summary' and 'action_items'.\n\n"
        f"Transcript:\n{transcript}"
    )

    response = client.chat.completions.create(
        model="myllm",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that formats meeting notes."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    message = response.choices[0].message.content

    try:
        parsed = MeetingNotes.parse_raw(message)
        return parsed.dict()
    except Exception:
        retry_prompt = "Please output valid JSON only.\n" + prompt
        retry_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Please output valid JSON only."},
                {"role": "user", "content": retry_prompt}
            ],
            temperature=0
        )
        retry_message = retry_response.choices[0].message.content
        return MeetingNotes.parse_raw(retry_message).dict()


Azure OpenAI Endpoint: https://eastus2.api.cognitive.microsoft.com/
OpenWeatherMap API Key: 6a8b0ac166a37e2b7a38e64416b3c3fe


## Step 3: Try It with a Sample Transcript

In [ ]:
sample_transcript = '''
Alice: We need to finalize the Q2 marketing campaign by next Monday.
Bob: I'll coordinate with the design team and send out the first draft by Friday.
Alice: Great, and I'll prepare the email templates over the weekend.
'''

output = extract_meeting_notes(sample_transcript)
print(output)


{'summary': 'The team discussed finalizing the Q2 marketing campaign by next Monday. Bob will coordinate with the design team to send out the first draft by Friday, while Alice will prepare the email templates over the weekend.', 'action_items': ['- Bob will coordinate with the design team and send out the first draft by Friday.', '- Alice will prepare the email templates over the weekend.']}


/tmp/ipython-input-22-2404821778.py:67: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  parsed = MeetingNotes.parse_raw(message)
/tmp/ipython-input-22-2404821778.py:68: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return parsed.dict()
